In [99]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import cv2
import time

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

from IPython import display
import csv

# splitting test data and validation data
all_data = np.loadtxt("COMP90086_2023_TLLdataset/train.csv",
                 delimiter=",", dtype=str)

left_imgs = all_data[:,0][1:]
right_imgs = all_data[:,1][1:]

## path for training data
data_dir = "COMP90086_2023_TLLdataset/train"



##split data into 20% testing and 60% training and 20% validation
left_train, left_test, right_train, right_test = train_test_split(left_imgs, right_imgs, random_state=104, test_size = 0.2, shuffle=True)
left_train, left_val, right_train, right_val = train_test_split(left_train, right_train, random_state=104, test_size = 0.25, shuffle=True)

print(left_val)

['aaa' 'aaz' 'abh' ... 'zzu' 'zzv' 'zzy']
['osr' 'mqw' 'cdx' ... 'fxo' 'goh' 'yqv']
['jkj' 'nqe' 'ole' 'nzd' 'iul' 'gci' 'cvh' 'nup' 'ffb' 'iym' 'rxr' 'mxo'
 'tco' 'ale' 'qft' 'oeh' 'ccl' 'bfi' 'jwu' 'ens' 'bdj' 'wci' 'olb' 'bkl'
 'vzs' 'rdj' 'dty' 'uvr' 'vck' 'dha' 'zlg' 'pgd' 'lde' 'puk' 'nuc' 'iuv'
 'tac' 'kcj' 'nuf' 'qru' 'thx' 'jsq' 'hvi' 'ewh' 'cjc' 'cvo' 'gmh' 'dbw'
 'ofk' 'xeh' 'wrv' 'hgh' 'uje' 'bnv' 'ruk' 'pof' 'vtp' 'sig' 'sfy' 'bgo'
 'one' 'gbj' 'vav' 'eng' 'csx' 'jag' 'abn' 'xbg' 'hpa' 'pkp' 'pdk' 'bdr'
 'vzm' 'epv' 'osj' 'vgt' 'aif' 'xwd' 'yqt' 'dbc' 'sxg' 'mak' 'eub' 'vui'
 'wcw' 'dcb' 'ufc' 'jow' 'msc' 'clp' 'efi' 'svo' 'jbi' 'zae' 'ejf' 'jlt'
 'cfz' 'sax' 'uvm' 'yjm' 'znf' 'hzs' 'zwe' 'ova' 'fmc' 'kzq' 'iak' 'uzs'
 'myy' 'tce' 'mxy' 'qzy' 'knn' 'gcb' 'mci' 'ykh' 'tor' 'suk' 'jco' 'dmu'
 'yiv' 'vqb' 'beo' 'caw' 'iix' 'awg' 'dcp' 'thi' 'nfs' 'oru' 'oee' 'jdh'
 'jxp' 'hse' 'blk' 'hsh' 'oti' 'tys' 'yrl' 'evc' 'hfi' 'sld' 'zue' 'xxj'
 'umj' 'bds' 'flf' 'unu' 'asx' 'czb' 'hb

In [125]:
##convert image names to gray scale image arrays:
def convert_left(array, data_dir):
    image_str = data_dir+"/left/"+array[0]+".jpg"
    img_array = cv2.imread(image_str, cv2.IMREAD_GRAYSCALE)
    ##img_array = cv2.resize(img_array, (40, 49))
    total = np.array([img_array])
    
    for val in array[1:]:
        image_str = data_dir+"/left/"+val+".jpg"
        img_array = cv2.imread(image_str, cv2.IMREAD_GRAYSCALE)
        ##img_array = cv2.resize(img_array, (40, 49))
        total = np.append(total, np.array([img_array]), axis = 0)
    return total

In [126]:
left_train_img_arrays = convert_left(left_train, data_dir)
left_val_img_arrays = convert_left(left_val, data_dir)
print(left_val_img_arrays.shape)

(400, 245, 200)


In [127]:
# Normalize and reshape the data
left_train_img_arrays = left_train_img_arrays.astype('float32') / 255.
left_val_img_arrays = left_val_img_arrays.astype('float32') / 255.


#x_left_train = np.reshape(left_train_img_arrays, (len(left_train_img_arrays), 32, 32, 1))
# x_test = np.reshape(test_data, (len(test_data), 28, 28, 1))

x_left_train = left_train_img_arrays.reshape((len(left_train_img_arrays), np.prod(left_train_img_arrays.shape[1:])))
x_left_val = left_val_img_arrays.reshape((len(left_val_img_arrays), np.prod(left_val_img_arrays.shape[1:])))

print("train_images shape:", x_left_train.shape)


train_images shape: (1200, 49000)


In [130]:
# This is the size of our encoded representations
latent_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(49000,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(latent_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(49000, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(latent_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

autoencoder.fit(x_left_train, x_left_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_left_val, x_left_val))

Epoch 1/20
5/5 [==============================] - 2s 359ms/step - loss: 0.6931 - accuracy: 0.0000e+00 - val_loss: 0.6912 - val_accuracy: 0.0000e+00
Epoch 2/20
5/5 [==============================] - 2s 329ms/step - loss: 0.6865 - accuracy: 0.0000e+00 - val_loss: 0.6728 - val_accuracy: 0.0000e+00
Epoch 3/20
5/5 [==============================] - 2s 333ms/step - loss: 0.6666 - accuracy: 0.0000e+00 - val_loss: 0.6548 - val_accuracy: 0.0000e+00
Epoch 4/20
5/5 [==============================] - 2s 332ms/step - loss: 0.6555 - accuracy: 0.0000e+00 - val_loss: 0.6517 - val_accuracy: 0.0000e+00
Epoch 5/20
2/5 [===========>..................] - ETA: 0s - loss: 0.6560 - accuracy: 0.0000e+00

KeyboardInterrupt: 